using 2021 census data, compare with 2022 budget to see if demographic has an impact by wards.

2021 census data by electorial wards: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E&SearchText=toronto%20centre&DGUIDlist=2023A000435109&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0

**"Median total income in 2020  among recipients ($)" is missng from the dataframe?**


In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [31]:
#add budget into dataframe
capital_budget = Path("Resources/2022-2031-capital-budget-and-plan-details.xlsx")
capital_budget_df = pd.read_excel(capital_budget, sheet_name = "2022-2031")

#loop through columns and clean up names
clean_column_names = [str(col).strip() for col in capital_budget_df.columns]
capital_budget_df.columns = clean_column_names

print(capital_budget_df.columns)

Index(['Program/Agency Name', 'Project Name', 'Sub-Project Name', '2022',
       '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031',
       'Total 10 Year', 'Ward Number', 'Ward', 'Category'],
      dtype='object')


In [32]:
#find and put unique ward names into list for future looping for profile data
ward_list = list(capital_budget_df["Ward"].unique())
ward_list.remove("City Wide")  #removing citywide from ward_list as it does not concern profile data
print(ward_list)

['York Centre ', 'Spadina-Fort York', 'Toronto Centre', 'Scarborough-Agincourt', 'Scarborough Centre', 'Eglinton-Lawrence', 'York South-Weston ', 'Scarborough-Rouge Park', 'Scarborough-North', 'Humber River-Black Creek', 'Davenport', 'Etobicoke North', 'Parkdale-High Park', 'Scarborough Southwest', 'Etobicoke-Lakeshore', 'Toronto-Danforth', 'Toronto-St. Pauls', 'Don Valley West', 'Scarborough-Guildwood', 'Beaches-East York', 'Don Valley North', 'University-Rosedale', 'Willowdale', 'Don Valley East', 'Etobicoke Centre']


In [34]:
#add up 2022 sum
sum_2022_by_ward = capital_budget_df.groupby("Ward")["2022"].sum().reset_index()
#new dataframe with clean summed values
clean_budget_df = pd.DataFrame({'Ward': sum_2022_by_ward['Ward'], 'Sum_2022': sum_2022_by_ward['2022']})
print("Updated Unique Ward Names:", ward_list)
print("\nClean Budget DataFrame:")
print(clean_budget_df)

Updated Unique Ward Names: ['York Centre ', 'Spadina-Fort York', 'Toronto Centre', 'Scarborough-Agincourt', 'Scarborough Centre', 'Eglinton-Lawrence', 'York South-Weston ', 'Scarborough-Rouge Park', 'Scarborough-North', 'Humber River-Black Creek', 'Davenport', 'Etobicoke North', 'Parkdale-High Park', 'Scarborough Southwest', 'Etobicoke-Lakeshore', 'Toronto-Danforth', 'Toronto-St. Pauls', 'Don Valley West', 'Scarborough-Guildwood', 'Beaches-East York', 'Don Valley North', 'University-Rosedale', 'Willowdale', 'Don Valley East', 'Etobicoke Centre']

Clean Budget DataFrame:
                        Ward    Sum_2022
0          Beaches-East York     1137.00
1                  City Wide  4230770.88
2                  Davenport     8957.00
3            Don Valley East     5190.00
4           Don Valley North    18480.00
5            Don Valley West     1535.00
6          Eglinton-Lawrence     3076.00
7           Etobicoke Centre    12213.00
8            Etobicoke North      198.00
9        Etob

In [12]:
#MOVE TO DIFFERENT PART? AFTER TAKING IN BUDGET?
#add census into dataframe

ward_profile = Path("Resources/98-401-X2021010_English_CSV_data.csv")
ward_profile_df = pd.read_csv(ward_profile, encoding="latin-1")

In [60]:
#GEO_NAME - but all the ones with "-" might be "--" or "- -"
#CHARACTERISTIC_NAME
#"    Median total income in 2020  among recipients ($)"
#"    Average total income in 2020 among recipients ($)"
#C1_COUNT_TOTAL
#"    Chinese"
#for geo: ward_list
#CHARACTERISTIC_NAME
income_data_row = ["    Median total income in 2020  among recipients ($)",
                  "    Average total income in 2020 among recipients ($)"
                  ]
minority_data_row = ["    South Asian",
                    "    Chinese",
                    "    Black",
                    "    Filipino",
                    "    Arab",
                    "    Latin American",
                    "    Southeast Asian",
                    "    West Asian",
                    "    Korean",
                    "    Japanese",
                    "    Visible minority, n.i.e.",
                    "    Multiple visible minorities",
                    "  Not a visible minority"
                    ]

In [63]:
#Filter rows based on "GEO_NAME" and clean the names
filtered_profile_df = ward_profile_df[ward_profile_df["GEO_NAME"].str.replace('--', '-') .isin(ward_list)]

#Filter rows based on "CHARACTERISTIC_NAME"
filtered_profile_df = filtered_profile_df[filtered_profile_df["CHARACTERISTIC_NAME"].isin(income_data_row + minority_data_row)]

#Create a new DataFrame with specific columns
clean_profile_df = filtered_profile_df[["GEO_NAME", "CHARACTERISTIC_NAME", "C1_COUNT_TOTAL"]]


#move characteristic name to columns, with C1_COUNT_TOTAL as the values
clean_profile_df = clean_profile_df.pivot(index="GEO_NAME", columns="CHARACTERISTIC_NAME", values="C1_COUNT_TOTAL").reset_index()

clean_profile_df["GEO_NAME"] = clean_profile_df["GEO_NAME"].str.replace('--', '-')

clean_profile_df.head()

#**NEED TO FIX HERE, income_data_row items are not here?

CHARACTERISTIC_NAME,GEO_NAME,Arab,Black,Chinese,Filipino,Japanese,Korean,Latin American,Multiple visible minorities,South Asian,Southeast Asian,"Visible minority, n.i.e.",West Asian,Not a visible minority
0,Beaches-East York,720.0,8260.0,5635.0,3565.0,635.0,705.0,2225.0,2280.0,12290.0,715.0,920.0,1260.0,69280.0
1,Davenport,660.0,7100.0,5525.0,3150.0,615.0,835.0,6130.0,2195.0,4820.0,2450.0,1215.0,525.0,69520.0
2,Don Valley East,3600.0,9880.0,7240.0,7175.0,680.0,1360.0,2290.0,2260.0,17285.0,810.0,1435.0,5520.0,34810.0
3,Don Valley North,2800.0,5720.0,32915.0,5680.0,620.0,4350.0,2485.0,2595.0,15560.0,1360.0,835.0,8945.0,28715.0
4,Don Valley West,1550.0,3835.0,12420.0,3510.0,525.0,1590.0,2150.0,1760.0,13410.0,615.0,750.0,5305.0,53610.0


In [64]:
#merge the two dataframes
merged_df = pd.merge(clean_budget_df, clean_profile_df, left_on="Ward", right_on="GEO_NAME", how="inner")
#drop GEO_NAME
merged_df = merged_df.drop("GEO_NAME", axis=1)

merged_df.head()

,Ward,Sum_2022,Arab,Black,Chinese,Filipino,Japanese,Korean,Latin American,Multiple visible minorities,South Asian,Southeast Asian,"Visible minority, n.i.e.",West Asian,Not a visible minority
0,Beaches-East York,1137.0,720.0,8260.0,5635.0,3565.0,635.0,705.0,2225.0,2280.0,12290.0,715.0,920.0,1260.0,69280.0
1,Davenport,8957.0,660.0,7100.0,5525.0,3150.0,615.0,835.0,6130.0,2195.0,4820.0,2450.0,1215.0,525.0,69520.0
2,Don Valley East,5190.0,3600.0,9880.0,7240.0,7175.0,680.0,1360.0,2290.0,2260.0,17285.0,810.0,1435.0,5520.0,34810.0
3,Don Valley North,18480.0,2800.0,5720.0,32915.0,5680.0,620.0,4350.0,2485.0,2595.0,15560.0,1360.0,835.0,8945.0,28715.0
4,Don Valley West,1535.0,1550.0,3835.0,12420.0,3510.0,525.0,1590.0,2150.0,1760.0,13410.0,615.0,750.0,5305.0,53610.0
